In [232]:
import torch
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [233]:
config = {}
config['patch_size'] = 3
config['number_of_patches_per_zone'] = 1
config['n_iter'] = 1

## Utils

In [234]:
def Patch(image, i, j, m):
    """ Returns a m x m patch taken from Image, centered at coordinates i,j """
    patch = image[:, :, i-m:i+m+1, j-m:j+m+1]
    return patch

In [235]:
def valid(coord, coord_limit, m):
    """ Makes sure the coordinates lie in a valid range (m corrects for the zero-padding) """
    coord = max(coord, m)
    coord = min(coord, coord_limit + m - 1)
    
    return coord

In [236]:
def warp(B, NNF_ab):
    """ Warp exprimes the content of image A but only with pixels sampled from image B """
    Warp = im2[:, NNF_ab[0,:,:], NNF_ab[1,:,:]]
    
    return Warp

## Similarity Metric

In [237]:
def EuclideanDistance(P1, P2):
    """ Returns euclidean distance between two patches """
    distance = torch.rsqrt(torch.sum((P1.float() - P2.float()) ** 2))
    return distance

## PatchMatch

In [238]:
def initializeNNF(h, w):
    """ 
    Randomly initializes NNF_ab
    - NNF_ab[:,i,j] is a 2D vector representing the coordinates x,y 
      so that B[0,:,x,y] is most similar to A[0,:,i,j] 
    """
    # Instanciates NNF_ab
    NNF_ab = np.zeros(shape=(2,h,w), dtype=np.int)
    
    # Fill in NNF_ab[:,:,0] contains the coordinates x (row)
    NNF_ab[0,:,:] = np.random.randint(low=0, high=h, size=(h,w))
        
    # Fill in NNF_ab[:,:,1] contains the coordinates y (column)
    NNF_ab[1,:,:] = np.random.randint(low=0, high=w, size=(h,w))
    
    # Turns it into an autograd.Variable and send it on the GPU
    if torch.cuda.is_available() and False:
        NNF_ab = Variable(torch.from_numpy(NNF_ab)).cuda()
    else:
        NNF_ab = Variable(torch.from_numpy(NNF_ab))
    
    return NNF_ab

In [239]:
def propagate(A, B, h, w, NNF_ab, i, j, shift):
    # Patch half-size
    m = int((config['patch_size'] - 1) / 2)

    # Extract the patch at coordinates i,j in A
    A_patch = Patch(A,i,j,m)

    # Extract the patch-match in B associated with current position, left neighbor and up neighbor in A
    A_current_patchMatch = Patch(B, NNF_ab[0,i,j], NNF_ab[1,i,j], m)
    A_leftNeighbor_patchMatch = Patch(B, NNF_ab[0,valid(i+shift,h,m),j], NNF_ab[1,valid(i+shift,h,m),j], m)
    A_upNeighbor_patchMatch = Patch(B, NNF_ab[0,i,valid(j+shift,w,m)], NNF_ab[1,i,valid(j+shift,w,m)], m)

    # Computes the distance between current patch and the three potential matches ()
    current_match = EuclideanDistance(A_patch, A_current_patchMatch)
    left_neighbor_match = EuclideanDistance(A_patch, A_leftNeighbor_patchMatch)
    up_neighbor_match = EuclideanDistance(A_patch, A_upNeighbor_patchMatch)
    
    # Looks up which match is the best. If best match is current match, nothing is changed
    best_match = np.argmin(np.array([current_match.data[0], left_neighbor_match.data[0], up_neighbor_match.data[0]]))


    if best_match == 1:
        # New patch-match in B is the same than for left-neighbor in A
        NNF_ab[:,i,j] = NNF_ab[:,valid(i+shift,h,m),j]

    if best_match == 2:
        # New patch-match in B is the same than for up-neighbor in A
        NNF_ab[:,i,j] = NNF_ab[:,i,valid(j+shift,w,m)]
    """
    if best_match == 1:
        # New patch-match in B based on left-neighbor's match
        NNF_ab[0,i,j] = NNF_ab[0,i+shift,j]
        NNF_ab[1,i,j] = valid(NNF_ab[1,i+shift,j]+1, w, m)

    if best_match == 2:
        # New patch-match in B based on up-neighbor's match
        NNF_ab[0,i,j] = valid(NNF_ab[0,i,j+shift]+1, h, m)
        NNF_ab[1,i,j] = NNF_ab[1,i,j+shift]
    """

    return NNF_ab

In [240]:
def randomSearch(A, B, h, w, NNF_ab, i, j):
    # Patch half-size
    m = int((config['patch_size'] - 1) / 2)

    max_step = 5
    while max_step < max(h,w):

        for k in range(config['number_of_patches_per_zone']):
            # The randomly sampled coordinates for the random patch-match
            [x, y] = NNF_ab[:,i,j].numpy() + np.random.randint(low=-max_step, high=max_step, size=(2,))

            # Makes sure that those coordinates lie within the limits of our image
            x = valid(x, h, m)
            y = valid(y, w, m)

            # Extract the patch around our current position in A
            A_patch = Patch(A,i,j,m)

            # Extract current patch-match and random patch-match in B
            A_current_patchMatch = Patch(B, NNF_ab[0,i,j], NNF_ab[1,i,j], m)
            A_random_patchMatch = Patch(B, x, y, m)

            # Computes how good both matches are
            current_match = EuclideanDistance(A_patch, A_current_patchMatch)
            random_match = EuclideanDistance(A_patch, A_random_patchMatch)

            # Looks up which match is the best. If best match is current match, nothing is changed
            best_match = np.argmin(np.array([current_match.data[0], random_match.data[0]]))

            if best_match == 1:
                NNF_ab[:,i,j] = torch.from_numpy(np.array([x,y]))

            max_step *= 3
                    
    return NNF_ab

In [241]:
def PatchMatch(image1, image2):
    """
    Computes the NNF function between image1 and image 2
    image 1 : an autograd.Variable of shape [batch, channels, heigth, width]
    image 2 : an autograd.Variable of shape [batch, channels, heigth, width]
    """
    
    # Makes sure the two images have the same size
    if image1.size() != image2.size():
        raise ValueError("The two images must have the same size.")
    
    # Heigth and Width of images
    [h,w] = image1.size()[2:]
     
    # Patch half-size
    m = int((config['patch_size'] - 1) / 2)
    
    # Randomly initializes NNF_ab
    NNF_ab = initializeNNF(h,w)
    
    # Zero-Pad images
    A = F.pad(image1, (m,m,m,m), mode='constant', value=0)
    B = F.pad(image2, (m,m,m,m), mode='constant', value=0)
    
    # Zero-Pad NNF_ab so its coordinate system as well as the values it contains remains consistent with images A and B
    NNF_ab += m
    NNF_ab = torch.squeeze(F.pad(NNF_ab.unsqueeze(0), (m,m,m,m), mode='constant', value=0)).data
    
    # Defines valid ranges (exclude the padded indexes)
    i_range = np.arange(h) + m
    j_range = np.arange(w) + m
    
    # Executes the PatchMatch algorithm n_iter times
    for step in range(config['n_iter']):
        print("Iteration {0} -----".format(step+1))
        if step%2 == 0:
            shift = 1
        else:
            shift = -1
        
        # For every valid pixel in the image
        for i in i_range:
            if (i+1)%100 == 0 : print("Row : {0}".format(i+1))
            for j in j_range:
                NNF_ab = propagate(A, B, h, w, NNF_ab, i, j, shift)
                NNF_ab = randomSearch(A, B, h, w, NNF_ab, i, j)
    
    NNF_final = NNF_ab[:, m:-m, m:-m]
    NNF_final -= m
    
    print("Done!")
    
    return NNF_final

## Demonstration

In [242]:
# Load the images
im1 = plt.imread(os.path.join("Images", "Image1.jpg"))
im2 = plt.imread(os.path.join("Images", "Image2.jpg"))

# Chagne images shape from [heigth, width, channels] to [channels, heigth, width]
imgs = [np.transpose(im1, axes=(2,0,1)), np.transpose(im2, axes=(2,0,1))]

# Turns them into autograd.Variables and send them on GPU (if available) and add a batch dimension
if torch.cuda.is_available() and False:
    print("Images on GPU")
    imgs_torch = [Variable(torch.from_numpy(img).unsqueeze(0).cuda()) for img in imgs]
else:
    print("Images on CPU")
    imgs_torch = [Variable(torch.from_numpy(img).unsqueeze(0)) for img in imgs]

# Runs the PatchMatch algorithm
%prun NNF_ab = PatchMatch(imgs_torch[0], imgs_torch[1])

Images on CPU
Iteration 1 -----
Row : 100
Row : 200
Row : 300
Row : 400
Done!
 

In [ ]:
# Displays the original images and resulting Warp
plt.figure(figsize=(16, 12))
plt.subplot(131)
plt.imshow(im1)
plt.title("Image A")
plt.axis("off")

plt.subplot(132)
plt.imshow(im2)
plt.title("Image B")
plt.axis("off")

plt.subplot(133)
plt.imshow(warp(im2, NNF))
plt.title("Warp")
plt.axis("off")
plt.show()


# Displays the NNF (mapping function)
zer = np.zeros(shape=(NNF.shape[0], NNF.shape[1], 1), dtype=NNF.dtype)
colorNNF = np.concatenate((np.expand_dims(NNF[:,:,0], 2), zer, np.expand_dims(NNF[:,:,1], 2)), axis=2)

plt.imshow(colorNNF.astype(np.uint8))
plt.title("NNF")
plt.axis("off")
plt.show()